In [1]:
# useful libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load dataset
df = pd.read_csv('data/clean_rideshare_data.csv')
df=df.head(100000)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         100000 non-null  int64  
 1   price              100000 non-null  float64
 2   distance           100000 non-null  float64
 3   surge_multiplier   100000 non-null  float64
 4   latitude           100000 non-null  float64
 5   longitude          100000 non-null  float64
 6   cab_type_Lyft      100000 non-null  float64
 7   cab_type_Uber      100000 non-null  float64
 8   name_Black         100000 non-null  float64
 9   name_Black SUV     100000 non-null  float64
 10  name_Lux           100000 non-null  float64
 11  name_Lux Black     100000 non-null  float64
 12  name_Lux Black XL  100000 non-null  float64
 13  name_Lyft          100000 non-null  float64
 14  name_Lyft XL       100000 non-null  float64
 15  name_Shared        100000 non-null  float64
 16  nam

In [18]:
# remove unnecessary data
df = df.drop(columns=['Unnamed: 0'],axis=1)

In [19]:
import pandas as pd
import folium
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

In [20]:
#  Unsupervised algorithm for high booking area
booking_data = df[['latitude', 'longitude']]  # Use relevant features for clustering
kmeans = KMeans(n_clusters=3)  # Specify the number of high booking areas you want to identify
kmeans.fit(booking_data)
high_booking_areas = kmeans.predict(booking_data)



In [21]:
#  Supervised algorithm for price prediction
X = df.drop(['price'],axis=1)  # Use relevant features for price prediction
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
price_predictions = regressor.predict(X_test)
price_mse = mean_squared_error(y_test, price_predictions)
r2_square = r2_score(y_test, price_predictions)


In [22]:
print(price_mse)
print(r2_square)

5.050120843493938
0.9350957360659906


In [23]:
#  Display the data on a map
map_obj = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

for _, row in df.iterrows():
    lat, lng = row['latitude'], row['longitude']
    marker_color = 'blue' if kmeans.predict([[lat, lng]]) in high_booking_areas else 'green'
    marker = folium.CircleMarker(location=[lat, lng], color=marker_color, radius=5, fill=True, fill_color=marker_color)
    marker.add_to(map_obj)

map_obj.save('map.html')
